### Crop images

In [1]:
import os
import json
from PIL import Image

# Paths
images_path = "../A-Dataset-and-Benchmark-for-Malaria-Life-Cycle-Classification-in-Thin-Blood-Smear-Images/IML_Malaria"
annotation_path = "../A-Dataset-and-Benchmark-for-Malaria-Life-Cycle-Classification-in-Thin-Blood-Smear-Images/annotations.json"
output_dir = "cell_dataset/"

# Make map for each class
classes = ["ring", "trophozoite", "schizont", "gametocyte", "red blood cell"]
for cls in classes:
    os.makedirs(os.path.join(output_dir, cls), exist_ok=True)

# Load annotations
with open(annotation_path) as f:
    ground_truth = json.load(f)

# Iterate through each image and its annotations
for entry in ground_truth:
    image_name = entry["image_name"]
    image_path = os.path.join(images_path, image_name)
    
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Bilde ikke funnet: {image_path}")
        continue

    for i, obj in enumerate(entry["objects"]):
        label = obj["type"]  
        if label not in classes:
            print(f"Unknown class: {label} — skipping")
            continue

        # Bounding box
        x = int(obj["bbox"]["x"])
        y = int(obj["bbox"]["y"])
        w = int(obj["bbox"]["w"])
        h = int(obj["bbox"]["h"])

        # Crop and save
        cropped = image.crop((x, y, x + w, y + h))
        save_path = os.path.join(output_dir, label, f"{image_name[:-4]}_{i}.png")
        if not os.path.exists(save_path):
            cropped.save(save_path)
        else:
            print(f"File {save_path} already exists, skipping save.")
print(" Done with cropping and saving images.")


Unknown class: difficult — skipping
Unknown class: difficult — skipping
Unknown class: difficult — skipping


KeyboardInterrupt: 

### Split data into train and valitadion set

In [ ]:
import random
import shutil 

# Paths
input_dir = "cell_dataset"        # Your full dataset with all images in class folders
output_dir = "data_split"         # Where train/val folders will be created
train_ratio = 0.8                 # 80% train, 20% validation split

# For reproducibility
random.seed(42)

# Create train and val directories
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# List all class folders
classes = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

for cls in classes:
    print(f"Splitting class: {cls}")
    cls_input_path = os.path.join(input_dir, cls)
    files = os.listdir(cls_input_path)
    random.shuffle(files)  # Shuffle files before splitting

    train_count = int(len(files) * train_ratio)
    train_files = files[:train_count]
    val_files = files[train_count:]

    # Create class folders inside train and val directories
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    # Copy training files
    for f in train_files:
        shutil.copy2(os.path.join(cls_input_path, f), os.path.join(train_dir, cls, f))

    # Copy validation files
    for f in val_files:
        shutil.copy2(os.path.join(cls_input_path, f), os.path.join(val_dir, cls, f))

print("Dataset split into train and validation sets!")

# After splitting, delete the original dataset folder to save space
shutil.rmtree("cell_dataset")
print("'cell_dataset' folder has been deleted to save space.")

FileNotFoundError: [Errno 2] No such file or directory: 'cell_dataset'

### 

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Image transformations (augmentasjon for trening, bare normalisering for validering)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],   # Standard ImageNet mean/std
                         std=[0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Last inn datasett
train_dataset = datasets.ImageFolder("data_split/train", transform=train_transforms)
val_dataset = datasets.ImageFolder("data_split/val", transform=val_transforms)

# DataLoader
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")


In [ ]:

import os
from PIL import Image
from transformers import ViTForImageClassification, ViTFeatureExtractor
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm



# === 1. Paths and constants ===
data_dir = "data_split"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 5
batch_size = 16
epochs = 5

# === 2. Load pretrained model and feature extractor ===
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=num_classes
)
model = model.to(device)

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# === 3. Custom dataset class ===
class MalariaDataset(Dataset):
    def __init__(self, root_dir, feature_extractor):
        self.root_dir = root_dir
        self.feature_extractor = feature_extractor
        self.classes = sorted(os.listdir(root_dir))
        self.paths = []
        self.labels = []
        for idx, cls in enumerate(self.classes):
            cls_folder = os.path.join(root_dir, cls)
            for img_name in os.listdir(cls_folder):
                self.paths.append(os.path.join(cls_folder, img_name))
                self.labels.append(idx)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image = Image.open(self.paths[idx]).convert("RGB")
        inputs = self.feature_extractor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].squeeze()  # Remove batch dim
        label = self.labels[idx]
        return pixel_values, label

# === 4. Load data ===
train_dataset = MalariaDataset(train_dir, feature_extractor)
val_dataset = MalariaDataset(val_dir, feature_extractor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# === 5. Optimizer and loss ===
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

# === 6. Training loop ===
for epoch in range(epochs):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        pixel_values, labels = batch
        pixel_values, labels = pixel_values.to(device), labels.to(device)

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch+1} - Train Loss: {total_loss/len(train_loader):.4f} - Train Accuracy: {acc:.4f}")

    # === 7. Evaluation ===
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for batch in val_loader:
            pixel_values, labels = batch
            pixel_values, labels = pixel_values.to(device), labels.to(device)
            outputs = model(pixel_values=pixel_values)
            preds = torch.argmax(outputs.logits, dim=-1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    print(f"Epoch {epoch+1} - Validation Accuracy: {val_acc:.4f}")


/Users/andreakristiane/MalariaClassification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/andreakristiane/MalariaClassification/.venv/lib/python3.12/site-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Training Epoch 1:  78%|███████▊  | 1493/1922 [1:57:53<1:28:40, 12.40s/it]  